In [65]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from datetime import date, timedelta

날짜 구하기

In [66]:
delta_day = timedelta(days=1)
delta_week = timedelta(weeks=1)

def get_dow(date_here):
    dow_list = ['mon','tue','wed','thu','fri','sat','sun']
    dow_index = date_here.weekday()
    return dow_list[dow_index]

def get_start_date():
    start_date = date.today() # 오늘 날짜 확인
    start_day = get_dow(start_date) # 오늘 요일을 시작 요일로  
    while start_day != 'sun': # 시작 요일이 일요일이 될 때까지
        start_date = start_date - delta_day # 날짜에서 하루를 뺌 
        start_day = get_dow(start_date) # 뺀 날짜의 요일을 시작 요일로 
    stop_date = start_date - delta_week
    return start_date, stop_date

def get_week(date_here):
    first_day = date_here.replace(day=1)
    week, weekday = divmod(date_here.day+ first_day.weekday() - 1, 7) #(몫, 나머지)
    return week+1
    

웹 스크래핑

In [129]:
url = 'http://kims4all.kr/login.aspx?ReturnUrl=%2fAPPFJ%2fDefault.aspx'

start_date, stop_date = get_start_date() # 시작 날짜, 끝 날짜 구해서 저장 
print(start_date, stop_date)

scrap_count = 0
error_count = 0

title_list=[]
definition_list=[]
example_list=[]  
note_list=[]

driver = webdriver.Chrome(executable_path=r'C:\Users\judya\Desktop\취준\chromedriver_win32\chromedriver.exe')
driver.get(url)

time.sleep(1)

# 로그인
driver.find_element_by_name("Login1$UserName").send_keys('박윤주')
driver.find_element_by_name("Login1$Password").send_keys('1234')
driver.find_element_by_name('Login1$LoginButton').click()
time.sleep(1)

# 상식 list 페이지 접근
driver.get('http://kims4all.kr/APPFJ/MediaTest/List.aspx?BoardName=generalKnowledge')

# 첫 아이템 접근
submit_element = driver.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_GridView1_ctl02_Label1"]')
submit_element.click()
time.sleep(1)

html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

item_num = int(soup.select_one('#ctl00_ContentPlaceHolder1_lblNum').get_text()) # 최초 페이지 넘버 저장 
datestamp = date.fromisoformat(soup.select_one('#ctl00_ContentPlaceHolder1_lblPostDate').get_text()[0:10]) # 최초 페이지 날짜 저장 

# 시작 아이템 페이지로 이동 
while datestamp > start_date:
    item_num -= 1
    driver.get(f'http://kims4all.kr/APPFJ/MediaTest/View.aspx?Num={item_num}&BoardName=generalKnowledge')
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    datestamp_new = date.fromisoformat(soup.select_one('#ctl00_ContentPlaceHolder1_lblPostDate').get_text()[0:10])
    if datestamp_new != date(1,1,1):
        datestamp = datestamp_new


# 스크래핑 
while datestamp > stop_date : 

    scrap_count += 1
    
    driver.get(f'http://kims4all.kr/APPFJ/MediaTest/View.aspx?Num={item_num}&BoardName=generalKnowledge')
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    datestamp_new = date.fromisoformat(soup.select_one('#ctl00_ContentPlaceHolder1_lblPostDate').get_text()[0:10])
    if datestamp_new != date(1,1,1):
        datestamp = datestamp_new
    
    # 표제어 가져오기
    title = soup.select_one('#ctl00_ContentPlaceHolder1_lblWordNPhrase').get_text()
    
    if title == '':
        
        error_count +=1
        print(f'{error_count}번째 삭제된 페이지 발견: {item_num}')
        item_num -=1
        
        continue

    title_list.append(title) 

    # 정의 가져오기 

    definition_pre = soup.find('span', {'id':"ctl00_ContentPlaceHolder1_lblSentence"}).find_all(text=True)
    time.sleep(.5)
    definition = '\n'.join(definition_pre)
    definition_list.append(definition) 

    # 예문 가져오기 
    example_pre = soup.find('span', {'id':"ctl00_ContentPlaceHolder1_lblNote"}).find_all(text=True)
    example = '\n'.join(example_pre)
    example_list.append(example)

    # 특기사항 가져오기 
    note_pre = soup.find('span', {'id':"ctl00_ContentPlaceHolder1_lblQuestions"}).find_all(text=True)
    time.sleep(.5)
    note = '\n'.join(note_pre)
    note_list.append(note) 
    
    item_num -=1
        
    if scrap_count%5 == 0 :
            print(f'상식 정리 진행 중... 정리된 상식 {scrap_count}개')
    
print(f'상식 정리 완료. 정리된 상식 총 {len(title_list)}개, 삭제된 페이지 {error_count}개')
    

1번째 삭제된 페이지 발견: 5991
2번째 삭제된 페이지 발견: 5989
상식 정리 진행 중... 정리된 상식 5개
상식 정리 완료. 정리된 상식 총 3개, 삭제된 페이지 2개


In [69]:
from docx import Document 
from docx.shared import Pt
from docx2pdf import convert

# install 방법: prompt에서 pip install python-docx
# pip install docx2pdf

In [70]:
# 월, 주 구하기 
if start_date.month != stop_date.month:
    start_first_day = start_date.replace(day=1)
    if start_first_day.weekday() > 1: 
        month = start_date.month
        week = get_week(start_date)
    else: 
        month = stop_date.month
        week = get_week(stop_date)

In [130]:
document = Document()
doc_num=0

style = document.styles['Normal']
style.font.name = '맑은 고딕'
style.font.size = Pt(10)

for i in range(len(title_list)):

    
    paragraph=document.add_paragraph(f'{doc_num+1}. '+title_list[doc_num])
    document.add_paragraph(definition_list[doc_num])
    
    if example_list[doc_num] != '':
        document.add_paragraph(example_list[doc_num])
    
    if note_list[doc_num] !='':
        document.add_paragraph(note_list[doc_num])
    document.add_paragraph()
    
    doc_num +=1

document.save(f'아프지 {month}월 {week}주차 시사상식.docx')

# 경로지정
input_file = 'C:\\Users\\judya\\Desktop\\취준\\상식\\아프지\\' + f'아프지 {month}월 {week}주차 시사상식.docx'
output_file = 'C:\\Users\\judya\\Desktop\\취준\\상식\\아프지\\' + f'아프지 {month}월 {week}주차 시사상식.pdf'    

convert(input_file, output_file)

print(f'{month}월 {week}주차 상식 파일 생성 완료')
